# Optimizing Data Performance: Leveraging Infer Schema and Schema Files in Databricks

In this project, we delve into an exploration of the merits and demerits associated with utilizing Infer Schema. We analyze the optimal scenarios for leveraging Infer Schema, shedding light on the intricacies that transpire behind the scenes when this feature is employed. Additionally, we scrutinize the drawbacks of Infer Schema, particularly its performance implications when handling large files. Furthermore, we propose strategies to mitigate performance issues encountered while utilizing Infer Schema in such scenarios.

### Infer Schema
Infer Schema is a feature commonly found in data processing frameworks and tools like Databricks that allows users to automatically deduce the schema or structure of their data without explicitly specifying it. 
Here's why it's beneficial:

- Automatic Schema Deduction: Infer Schema analyzes the data and determines the data types and structure of each column based on the actual data content. This eliminates the need for manual schema definition, saving time and effort.
- Flexibility and Agility: Especially in scenarios where the structure of the data may evolve over time, Infer Schema provides flexibility. As new columns or changes in data types occur, the schema can dynamically adjust without manual intervention.
- Reduced Error-Prone Manual Work: Manual schema definition can be error-prone, especially for large datasets with numerous columns. Infer Schema helps avoid mistakes that could lead to incorrect data processing.
- Ease of Onboarding: For new datasets or when working with unfamiliar data sources, Infer Schema provides a quick way to understand the structure of the data without requiring prior knowledge or documentation.
- Streamlined Development Process: By eliminating the need to define schemas manually, developers and data engineers can focus more on data analysis, transformation, and building pipelines, accelerating the development process.

When loading data without a predefined schema, InferSchema performs a scan of the dataset to automatically infer the structure and data types of each column. This process typically involves sampling a portion of the data to make initial assumptions about the data types, lengths, and formats present within the dataset. Here's what happens behind the scenes:  
**Data Sampling:** InferSchema initially samples a portion of the dataset to analyze the values and their characteristics. This sampling process helps in making preliminary guesses about the data types and structure.  
**Type Inference:** Based on the sampled data, InferSchema infers the data types for each column. For example, it might infer that a column predominantly contains numeric values, so it assigns a numeric data type accordingly.  
**Schema Generation:** After inferring the data types, InferSchema generates a schema definition for the dataset. This schema includes the inferred data types and any additional metadata such as column names.  


While InferSchema offers convenience and flexibility, it can encounter challenges, especially when dealing with very large datasets:  
**Performance Overhead:** In the case of large datasets, the process of inferring the schema can be computationally expensive and time-consuming. This overhead increases proportionally with the size of the dataset, potentially leading to longer data loading times and increased resource consumption.  
**Memory Consumption:** When processing large datasets, InferSchema may need to load significant portions of the data into memory for sampling and analysis. This can strain system resources, particularly in environments with limited memory capacity, potentially leading to performance degradation or out-of-memory errors.  
**Accuracy Limitations:** InferSchema's accuracy heavily depends on the quality and representativeness of the sampled data. With very large datasets, sampling a small portion may not accurately capture the diversity and complexity of the entire dataset, leading to potential inaccuracies in schema inference.  


To mitigate these issues when working with large datasets, it's advisable to consider alternative approaches such as providing a predefined schema file. This allows for more efficient and accurate schema definition without the need for costly schema inference operations, thereby improving overall performance and reliability, particularly in resource-constrained environments.

For this demo we are going to use the one of databricks dataset.   

#### Dataset details
- dataset Name: sai-summit-2019-sf
- Folder Path: dbfs:/databricks-datasets/sai-summit-2019-sf/
- Total File(s) Size (GB): 1.06
- Total File Count: 3
- File Types and Counts: {'md': 1, 'csv': 1}

##### creating temmporary view using inferSchema on a 1GB file

In [0]:
%sql
create or replace temporary view tableA 
using csv 
options 
(
  path = "/databricks-datasets/sai-summit-2019-sf/*.csv", 
  header = True, 
  inferSchema = True, 
  sep = ','
)

It took 42.13 seconds to create the temporary view tableA

In [0]:
%sql
select count(1) from tableA 

count(1)
4380660


##### creating a temporary view using inferSchema using schemaFile on the same 1GB File

In [0]:
%python
from pyspark.sql.types import *

# Define the mapping of Spark data types to string representations
datatype_mapping = {
    StringType(): "string", 
    IntegerType(): "int",
    LongType(): "long",
    DoubleType(): "double",
    FloatType(): "float",
    BooleanType(): "boolean",
    TimestampType(): "timestamp",
    DateType(): "date"
}

# Read the data with inferSchema=True
df = spark.read.option("samplingRatio", 0.01).csv("/databricks-datasets/sai-summit-2019-sf/*.csv", inferSchema=True, header=True, sep=',')

# Create the schema string
schema_string = ", ".join([f"{field.name} {datatype_mapping.get(field.dataType, 'string')}" for field in df.schema.fields])

# Create or replace temporary view with the inferred schema
df.createOrReplaceTempView("tableB")

It took only 11.43 seconds when compared to the first result. 

In [0]:
%sql
select count(1) from tableB

count(1)
4380660


We have increased the performance by **72.86%** with the help of schemaFile.